<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/YouTubeToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install -U openai-whisper

In [ ]:
pip install pytubefix

In [ ]:
pip install moviepy

In [ ]:
pip install google-api-python-client

In [ ]:
import html
import os
import datetime
import csv
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
from pytubefix import YouTube
from moviepy.editor import *
import whisper

#-------↓パラメータ入力↓-------

developerKey = ""
channel_id = "UCmuo6-sGvNxqT2NF7SWKClg"

#-------↑パラメータ入力↑-------

youtube = build("youtube", "v3", developerKey=developerKey)

videos = [] #videoURLを格納する配列
dt_now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

def youtube_search(pagetoken, st, ed):

    search_response  = youtube.search().list(
        part = "snippet",
        channelId = channel_id,
        maxResults = 50,
        type = "video",
        publishedAfter = st, #'2013-01-01T00:00:00Z',
        publishedBefore = ed, #'2014-01-01T00:00:00Z',
        order = "date",
        pageToken = pagetoken
    ).execute()

    print(search_response["pageInfo"]["totalResults"])
    for search_result in search_response.get("items", []):

        if search_result["id"]["kind"] == "youtube#video":
            publishedAt = search_result["snippet"]["publishedAt"]
            title = html.unescape(search_result["snippet"]["title"])
            id = search_result["id"]["videoId"]
            url = 'https://www.youtube.com/watch?v=%s' % search_result["id"]["videoId"]

            videos.append([publishedAt, title, id, url])
            youtube_to_text(id, url)
    try:
        nextPagetoken = search_response["nextPageToken"]
        youtube_search(nextPagetoken, st, ed)
    except:
        return

def youtube_to_text(id, url):

    # YouTube動画をロード
    yt = YouTube(url)

    # 最高品質のオーディオストリームを選択
    audio_stream = yt.streams.filter(only_audio=True).first()

    # オーディオを一時ファイルとしてダウンロード
    temp_file = audio_stream.download()

    # 出力ファイル名
    audiofile = id + ".mp3"

    # MoviePyを使用してオーディオをMP3に変換
    audio_clip = AudioFileClip(temp_file)
    audio_clip.write_audiofile("/content/drive/MyDrive/" + audiofile, codec="libmp3lame")

    # 一時ファイルを削除
    os.remove(temp_file)

    # 出力ファイル名（適宜変更する）
    outputfile = id + ".txt"

    # 処理
    model = whisper.load_model("small", device="cpu")
    # model = whisper.load_model("medium", device="cuda")
    result = model.transcribe("/content/drive/MyDrive/" + audiofile, language="ja", verbose=True)

    # セグメントごとに改行してテキストを取得
    segments = result["segments"]
    transcript = "\n".join(segment["text"] for segment in segments)

    # txtへ書き出し
    with open("/content/drive/MyDrive/" + outputfile, "w", encoding='utf-8_sig') as f:
        f.write(transcript)

# 2023年1月1日から（ここを適宜変更）
dt = datetime.datetime(2023, 1, 1, 0, 0)
#
for i in range(1, 21):
    youtube_search('', dt.isoformat()+'Z', (dt + relativedelta(months=1)).isoformat()+'Z')
    dt = dt + relativedelta(months=1)

In [ ]:
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/drive/MyDrive/download.zip /content/drive/MyDrive

In [ ]:
# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download("/content/drive/MyDrive/download.zip")